<!-- TABS -->
# Retrieval augmented generation

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = False

In [2]:
from superduper import superduper

db = superduper('mongomock:///test_db')

2024-Oct-16 09:31:34.98| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2024-Oct-16 09:31:35.03| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:73   | Building Data Layer
2024-Oct-16 09:31:35.03| INFO     | Duncans-MBP.fritz.box| superduper.base.build:177  | Configuration: 
 +---------------+----------------------+
| Configuration |        Value         |
+---------------+----------------------+
|  Data Backend | mongomock:///test_db |
+---------------+----------------------+


In [3]:
COLLECTION_NAME = '<var:table_name>'

<!-- TABS -->
## Get useful sample data

In [4]:
import json

with open('data.json', 'r') as f:
    data = json.load(f)
data = [{'x': r} for r in data]

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [5]:
if APPLY:
    from superduper import Document
    
    ids = db.execute(db[COLLECTION_NAME].insert([Document(r) for r in data]))

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [6]:
from superduper import model

CHUNK_SIZE = 200

@model(signature='singleton', example='test ' * 50)
def chunker(text):
    text = text.split()
    chunks = [' '.join(text[i:i + CHUNK_SIZE]) for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [7]:
from superduper import Listener

upstream_listener = Listener(
    model=chunker,
    select=db[COLLECTION_NAME].select(),
    key='x',
    identifier='chunker',
    flatten=True,
)

In [8]:
if APPLY:
    db.apply(upstream_listener)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [9]:
import os
from superduper import vector

os.environ['OPENAI_API_KEY'] = 'sk-<secret>'
from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(identifier='text-embedding-ada-002', datatype=vector(shape=(1024,)))

Sentence-transformers

In [10]:
from superduper import vector
import sentence_transformers
from superduper_sentence_transformers import SentenceTransformer

sentence_transformers_embedding = SentenceTransformer(
    identifier="sentence-transformers-embedding",
    model="BAAI/bge-small-en",
    datatype=vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
    predict_kwargs={"show_progress_bar": True},
)

/Users/dodo/.pyenv/versions/3.10.13/envs/superduper-3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
from superduper.components.model import ModelRouter

embedding_model = ModelRouter(
    'embedding',
    models={'openai': openai_embedding, 'sentence_transformers': sentence_transformers_embedding},
    model='<var:embedding_model>',
    example='this is a test',
)

## Create vector-index

In [12]:
from superduper import VectorIndex, Listener

vector_index_name = 'vector-index'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs].select(),
        model=embedding_model,
        identifier='embedding-listener',
        upstream=[upstream_listener],
    )
)

In [13]:
if APPLY:
    db.apply(vector_index)

<!-- TABS -->
## Build LLM

In [14]:
from superduper_openai import OpenAIChatCompletion

llm_openai = OpenAIChatCompletion(identifier='llm-openai', model='gpt-3.5-turbo')

In [15]:
from superduper_anthropic import AnthropicCompletions
import os

os.environ["ANTHROPIC_API_KEY"] = "sk-ant-<secret>"

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}

llm_anthropic = AnthropicCompletions(identifier='llm-vllm', model='claude-2.1', predict_kwargs=predict_kwargs)

In [16]:
from superduper_vllm import VllmCompletion

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}

llm_vllm = VllmCompletion(
    identifier="llm-vllm",
    vllm_params={
        'model': 'TheBloke/Mistral-7B-Instruct-v0.2-AWQ',
        "gpu_memory_utilization": 0.7,
        "max_model_len": 1024,
        "quantization": "awq",
    },
    predict_kwargs=predict_kwargs,
)

2024-Oct-16 09:31:40.05| INFO     | Duncans-MBP.fritz.box| superduper_vllm.model:31   | Setting num_gpus to 1


In [17]:
# # !huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
# from superduper_llamacpp.model import LlamaCpp

# llm_llamacpp = LlamaCpp(identifier="llm-llamacpp", model_name_or_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf")

In [18]:
llm = ModelRouter(
    'llm',
    models={
        'openai': llm_openai,
        'anthropic': llm_anthropic,
        'vllm': llm_vllm,
        # 'llamacpp': llm_llamacpp,
    },
    model='<var:llm_model>',
)

## Answer question with LLM

In [19]:
from superduper import model
from superduper.components.model import RAGModel

prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

rag = RAGModel(
    'rag-model',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm,
)

In [20]:
if APPLY:
    db.apply(rag)

In [21]:
if APPLY:
    rag.predict('Tell me about vector-search')

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [22]:
from superduper import Application

app = Application(
    'rag-app',
    components=[
        upstream_listener,
        vector_index,
        rag,
    ]
)

In [23]:
if APPLY:
    db.apply(app)

You can now load the model elsewhere and make predictions using the following command.

## Create template

In [24]:
from superduper import Template

template = Template(
    'rag',
    template=app,
    data=data,
    substitutions={'docs': 'table_name'},
    template_variables=['llm_model', 'embedding_model', 'table_name'],
    types={
        'llm_model': {
            'type': 'str',
            'choices': ['openai', 'anthropic', 'vllm', 'llamacpp'],
            'default': 'openai',
        },
        'embedding_model': {
            'type': 'str',
            'choices': ['openai', 'sentence_transformers'],
            'default': 'openai',
        },
        'table_name': {
            'type': 'str',
            'default': '_sample_rag'
        }
    }
)

2024-Oct-16 09:31:40.07| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:471  | Leaf listener:chunker already exists
2024-Oct-16 09:31:40.07| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:471  | Leaf model:chunker already exists
2024-Oct-16 09:31:40.07| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:471  | Leaf datatype:dill_lazy already exists
2024-Oct-16 09:31:40.07| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:471  | Leaf var-table-name-select already exists
2024-Oct-16 09:31:40.08| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:471  | Leaf datatype:vector[1024] already exists
2024-Oct-16 09:31:40.08| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:471  | Leaf model:llm-vllm already exists


In [25]:
template.export('.')